In [24]:
# 모듈 불러오기
import pandas as pd
import numpy as np
import time
import re

from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 1. Main Page

In [4]:
# Selenium 웹 드라이버 설정
driver = webdriver.Chrome()

# IMDb Bridgerton 페이지로 이동
driver.get("https://www.imdb.com/title/tt8740790/")

In [15]:
# 한국 이름
driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-b7c53eda-0.dUpRPQ > h1 > span').text

'브리저튼'

In [18]:
# 원제
driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-b7c53eda-0.dUpRPQ > div').text[16:]

'Bridgerton'

In [9]:
# 전체 시즌의 평점
driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK').text

'7.4'

In [12]:
# 전체 시즌의 평점을 매긴 사람 수
driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-3.gPVQxL').text

'178K'

In [14]:
# Watchlist에 추가한 사람 수
driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > div:nth-child(3) > div:nth-child(2) > div > button.ipc-split-button__btn > div > div.sc-b23676b3-3.bRgISf').text

'18만명의 사용자가 추가'

In [20]:
# 유저 리뷰 수
driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > ul > li:nth-child(1) > a > span > span.score').text

'1.9K'

In [21]:
# 전문가 리뷰 수
driver.find_element(By.CSS_SELECTOR, '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > ul > li:nth-child(2) > a > span > span.score').text

'131'

In [ ]:
# 브라우저 닫기
driver.quit()

## 2. Episode

In [25]:
# 현재 페이지 내의 정보를 저장
raw = driver.page_source

# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

### (1) 전체 시즌
- beautifulsoup 상에서는 총 5개라고 뜸
- 1개는 그냥 의미없는 데이터인듯. 주유소 크롤링할 때 시/도 데이터처럼..
- 사이트 상에서는 1, 2, 3, 4 시즌이 표기되는데, 현재 촬영이 진행되고 있는 것은 3시즌. 4시즌은 아마 제작 예정이라 넣어놓은듯?
- -> 방영 일자를 함께 크롤링하고, 방영 일자의 연도가 2024거나 연도 크롤링 시 에러가 발생한다면 크롤링을 중지하는 방법

In [57]:
# 전체 시즌 개수
season_count = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-56501f3b-0.iZwhod > div.ipc-tabs.ipc-tabs--base.ipc-tabs--align-left.ipc-tabs--display-chip.ipc-tabs--inherit > ul')[0]
len(season_count)

5

### (2) 시즌 1
- len으로 season1의 에피소트 개수 가져오기
- for문으로 for i in range(len(season1)): 해서 article:nth-child(i)를 1부터 8까지 증가시켜주면 될듯...

In [34]:
# 시즌 1
season1 = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW')[0]
len(season1)

8

In [39]:
# 시즌 1 1화 제목
season1.select_one('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW > article:nth-child(1) > div > div > div.sc-ccd6e31b-4.eMYVLm > div.sc-ccd6e31b-5.cXcoHr > h4 > div > a > div').text

'S1.E1 ∙ Diamond of the First Water'

In [43]:
# 시즌 1 1화 방영 일자
season1.select_one('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW > article:nth-child(1) > div > div > div.sc-ccd6e31b-4.eMYVLm > div.sc-ccd6e31b-5.cXcoHr > span').text

'Fri, Dec 25, 2020'

In [42]:
# 시즌 1 1화 소개
season1.select_one('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW > article:nth-child(1) > div > div > div.sc-ccd6e31b-4.eMYVLm > div.sc-ccd6e31b-11.hJduta > div > div > div > div').text

"Daphne Bridgerton debuts on London's marriage market as a new gossip sheet sets high society atwitter and Simon, the eligible Duke of Hastings, returns to town."

In [50]:
# 시즌 1 1화 별점
season1.select_one('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW > article:nth-child(1) > div > div > div.sc-ccd6e31b-4.eMYVLm > div.sc-ccd6e31b-12.kiyCDq > div > span').text

'7.2/10\xa0(3.8K)'

In [38]:
# 시즌 1 2화 제목
season1.select_one('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2) > section.sc-33cc047c-0.guTCiW > article:nth-child(2) > div > div > div.sc-ccd6e31b-4.eMYVLm > div.sc-ccd6e31b-5.cXcoHr > h4 > div > a > div').text

# article:nth-child(2) 로 바꾸면 2화가 크롤링 되는 것을 확인

'S1.E2 ∙ Shock and Delight'

## 3. Cast & Crew

In [ ]:
# 주소
https://www.imdb.com/title/tt8740790/fullcredits

In [58]:
# 현재 페이지 내의 정보를 저장
raw = driver.page_source

# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

In [85]:
# 감독
names = soup.select('#fullcredits_content > table:nth-child(2)')[0].find_all('td', class_='name')
for name in names:
    person_name = name.find('a').text.strip()
    print(person_name)

Tom Verica
Tricia Brock
Sheree Folkson
Alrick Riley
Julie Anne Robinson
Cheryl Dunye
Alex Pillai
Andrew Ahn
Bille Woodruff


In [86]:
# 작가
names = soup.select('#fullcredits_content > table:nth-child(4)')[0].find_all('td', class_='name')
for name in names:
    person_name = name.find('a').text.strip()
    print(person_name)

Julia Quinn
Chris Van Dusen
Chris Van Dusen
Chris Van Dusen
Geetika Lizardi
Geetika Lizardi
Geetika Lizardi
Jess Brownell
Jess Brownell
Abby McDonald
Abby McDonald
Joy C. Mitchell
Joy C. Mitchell
Daniel Robinson
Sarah L. Thompson
Leila Cohan
Sarah Dollard
Janet Lin
Jay Ross
Oliver Goldstick
Lou-Lou Igbokwe
Lauren Gamble
Azia Squire
Eli Wilson Pelton


In [119]:
# 배우
cast_list = soup.find('table', class_='cast_list')
actors = cast_list.find_all('td', class_='primary_photo')

for actor in actors:
    actor_name = actor.find('a').img['title']
    print(actor_name)

Nicola Coughlan
Claudia Jessie
Luke Newton
Julie Andrews
Ruth Gemmell
Luke Thompson
Polly Walker
Jessica Madsen
Adjoa Andoh
Bessie Carter
Florence Hunt
Will Tilston
Joanna Bobin
Jonathan Bailey
Golda Rosheuvel
Hugh Sachs
Kathryn Drysdale
Lorraine Ashbourne
Harriet Cains
Martins Imhangbe
Phoebe Dynevor
Emma Naomi
Joanne Henry
Simone Ashley
Geraldine Alexander
Ruby Barker
Charithra Chandran
Shelley Conn
Ben Miller
Regé-Jean Page
Rupert Young
Lorn Macdonald
Molly McGlynn
Hannah Dodd
Victor Alli
Jason Barnett
Dominic Coleman
Bert Seymour
Calam Lynch
Caleb Obediah
Oli Higginson
Sabrina Bartlett
Ruby Stokes
Esme Coy
Julian Ovenden
Daniel Francis
Hannah New
Sarah Junillon
James Phoon
Jessie Baek
Simon Ludders
Melissa Advani
Rosa Hesmondhalgh
Ray MacAllan
Joe Barnes
James Bryan
Al Nedjari
James Fleet
Ian Attard
Sam Phillips
Gabrielle Oke
Jorden Myrie
Freddie Stroma
Ned Porteous
Emily Barber
Molly Jackson-Shaw
Thomas Flynn
James Matthews
Joseph Macnab
Sandra Teles
Naomi Preston-Low
Banita Sandh

270

## 4. Company Credits

In [ ]:
# 주소
https://www.imdb.com/title/tt8740790/companycredits

In [121]:
# 현재 페이지 내의 정보를 저장
raw = driver.page_source

# BeautifulSoup을 이용하여 html 파싱
soup = BeautifulSoup(raw, 'html.parser')

In [126]:
# 제작사
production_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
production_companies = production_list.find_all(class_="ipc-metadata-list-item__label--link")

for company in production_companies:
    print(company.text)

Shondaland
CVD Productions
